In [ ]:
! pip install lakefs-client

  Using cached lakefs_client-1.44.0-py3-none-any.whl.metadata (41 kB)
Using cached lakefs_client-1.44.0-py3-none-any.whl (378 kB)

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# import lakefs_client
# from lakefs_client import Configuration
# from lakefs_client.client import repositories_api, branches_api, objects_api
import pandas as pd
import numpy as np

In [ ]:
# Replace with your actual values
lakefs_endpoint = "http://localhost:8000"
access_key = "AKIA..."
secret_key = "abc123..."

configuration = Configuration()
configuration.username = access_key
configuration.password = secret_key
configuration.host = lakefs_endpoint

client = LakeFSClient(configuration)

In [10]:
def clean_data(raw_data):
    clean_data = raw_data.copy()
    # Remove not relevant columns
    clean_data = clean_data.dropna(subset=['region','age','weight','height','howlong','gender','eat', \
                               'train','background','experience','schedule','howlong', \
                               'deadlift','candj','snatch','backsq','experience',\
                               'background','schedule','howlong'])
    clean_data = clean_data.drop(columns=['affiliate','team','name','athlete_id','fran','helen','grace',\
                              'filthy50','fgonebad','run400','run5k','pullups','train'])

    # Remove Outliers

    clean_data = clean_data[clean_data['weight'] < 1500]
    clean_data = clean_data[clean_data['gender'] != '--']
    clean_data = clean_data[clean_data['age'] >= 18]
    clean_data = clean_data[(clean_data['height'] < 96) & (clean_data['height'] > 48)]

    clean_data = clean_data[(clean_data['deadlift'] > 0) & (clean_data['deadlift'] <= 1105)|((clean_data['gender'] == 'Female') \
                 & (clean_data['deadlift'] <= 636))]
    clean_data = clean_data[(clean_data['candj'] > 0) & (clean_data['candj'] <= 395)]
    clean_data = clean_data[(clean_data['snatch'] > 0) & (clean_data['snatch'] <= 496)]
    clean_data = clean_data[(clean_data['backsq'] > 0) & (clean_data['backsq'] <= 1069)]

    # Clean Survey Data

    decline_dict = {'Decline to answer|': np.nan}
    clean_data = clean_data.replace(decline_dict)
    clean_data = clean_data.dropna(subset=['background','experience','schedule','howlong','eat'])
    return clean_data

In [11]:
orig_data = pd.read_csv('athletes.csv')
clean_data = clean_data(orig_data)
clean_data.to_csv('athletes.csv')

In [6]:
orig_data.columns

Index(['athlete_id', 'name', 'region', 'team', 'affiliate', 'gender', 'age',
       'height', 'weight', 'fran', 'helen', 'grace', 'filthy50', 'fgonebad',
       'run400', 'run5k', 'candj', 'snatch', 'deadlift', 'backsq', 'pullups',
       'eat', 'train', 'background', 'experience', 'schedule', 'howlong'],
      dtype='object')